In [2]:
import numpy as np
import pandas as pd
from IPython.display import display
pd.options.display.float_format = "{:.6f}".format

#### The differential evolution algorithm

In [ ]:
def differential_evolution(func, rnge, num_individuals=30, iterations=50, f=0.8, cr=0.9):
    lower, upper = rnge
    dim = 2
    population = np.random.uniform(lower, upper, size=(num_individuals, dim))
    values = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, population)
    for _ in range(iterations):
        new_population = population.copy()
        new_values = values.copy()
        for i in range(num_individuals):
            candidate_indices = np.delete(np.arange(num_individuals), i)
            r1, r2, r3 = np.random.choice(candidate_indices, 3, replace=False)
            donor = population[r3] + f * (population[r1] - population[r2])
            donor = np.clip(donor, lower, upper)
            trial = population[i].copy()
            j_rand = np.random.randint(dim)
            for j in range(dim):
                if np.random.uniform() < cr or j == j_rand:
                    trial[j] = donor[j]
            trial_value = func(tuple(trial))
            if trial_value <= values[i]:
                new_population[i] = trial
                new_values[i] = trial_value
        population = new_population
        values = new_values
    best_idx = np.argmin(values)
    best_position = population[best_idx]
    best_value = float(values[best_idx])
    return float(best_position[0]), float(best_position[1]), best_value

In [ ]:
def particle_swarm_optimization(func, rnge, pop_size=30, iterations=50, w=0.7, c1=1.5, c2=1.5):
    lower, upper = rnge
    dim = 2
    v_max = 0.2 * (upper - lower)
    positions = np.random.uniform(lower, upper, size=(pop_size, dim))
    velocities = np.random.uniform(-v_max, v_max, size=(pop_size, dim))
    fitness = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, positions)
    pbest_positions = positions.copy()
    pbest_values = fitness.copy()
    gbest_idx = np.argmin(fitness)
    gbest_position = positions[gbest_idx].copy()
    gbest_value = float(fitness[gbest_idx])
    for _ in range(iterations):
        r1 = np.random.uniform(size=(pop_size, dim))
        r2 = np.random.uniform(size=(pop_size, dim))
        velocities = (
            w * velocities
            + c1 * r1 * (pbest_positions - positions)
            + c2 * r2 * (gbest_position - positions)
        )
        velocities = np.clip(velocities, -v_max, v_max)
        positions = positions + velocities
        positions = np.clip(positions, lower, upper)
        fitness = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, positions)
        improved = fitness < pbest_values
        pbest_positions[improved] = positions[improved]
        pbest_values[improved] = fitness[improved]
        best_particle = np.argmin(pbest_values)
        if pbest_values[best_particle] < gbest_value:
            gbest_value = float(pbest_values[best_particle])
            gbest_position = pbest_positions[best_particle].copy()
    return float(gbest_position[0]), float(gbest_position[1]), gbest_value

In [ ]:
def soma_all_to_one(func, rnge, pop_size=30, prt=0.4, path_length=3.0, step=0.11, migrations=50):
    lower, upper = rnge
    dim = 2
    positions = np.random.uniform(lower, upper, size=(pop_size, dim))
    fitness = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, positions)
    gbest_idx = np.argmin(fitness)
    gbest_position = positions[gbest_idx].copy()
    gbest_value = float(fitness[gbest_idx])
    for _ in range(migrations):
        gbest_idx = np.argmin(fitness)
        leader_position = positions[gbest_idx].copy()
        for i in range(pop_size):
            if i == gbest_idx:
                continue
            start_position = positions[i].copy()
            best_local_pos = start_position.copy()
            best_local_val = float(fitness[i])
            prt_vector = (np.random.uniform(size=dim) < prt).astype(float)
            if not prt_vector.any():
                prt_vector[np.random.randint(dim)] = 1.0
            t_values = np.arange(step, path_length + step, step)
            for t in t_values:
                candidate = start_position + (leader_position - start_position) * t * prt_vector
                candidate = np.clip(candidate, lower, upper)
                value = float(func(tuple(candidate)))
                if value < best_local_val:
                    best_local_val = value
                    best_local_pos = candidate.copy()
                if value < gbest_value:
                    gbest_value = value
                    gbest_position = candidate.copy()
            positions[i] = best_local_pos
            fitness[i] = best_local_val
        gbest_idx = np.argmin(fitness)
        if fitness[gbest_idx] < gbest_value:
            gbest_value = float(fitness[gbest_idx])
            gbest_position = positions[gbest_idx].copy()
    return float(gbest_position[0]), float(gbest_position[1]), gbest_value

In [ ]:
def firefly_algorithm(
        func,
        rnge,
        pop_size=30,
        generations=50,
        beta0=1.0,
        gamma=1.0,
        alpha=0.25,
        alpha_decay=0.97):
    lower, upper = rnge
    dim = 2
    positions = np.random.uniform(lower, upper, size=(pop_size, dim))
    fitness = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, positions)
    best_idx = np.argmin(fitness)
    best_position = positions[best_idx].copy()
    best_value = float(fitness[best_idx])
    current_alpha = alpha
    for _ in range(generations):
        for i in range(pop_size):
            for j in range(pop_size):
                if fitness[j] < fitness[i]:
                    difference = positions[j] - positions[i]
                    distance_sq = np.dot(difference, difference)
                    beta = beta0 * np.exp(-gamma * distance_sq)
                    random_step = np.random.uniform(-0.5, 0.5, size=dim) * (upper - lower)
                    candidate = positions[i] + beta * difference + current_alpha * random_step
                    candidate = np.clip(candidate, lower, upper)
                    candidate_value = float(func(tuple(candidate)))
                    positions[i] = candidate
                    fitness[i] = candidate_value
                    if candidate_value < best_value:
                        best_value = candidate_value
                        best_position = candidate.copy()
        current_alpha *= alpha_decay
        current_best_idx = np.argmin(fitness)
        if fitness[current_best_idx] < best_value:
            best_value = float(fitness[current_best_idx])
            best_position = positions[current_best_idx].copy()
    return float(best_position[0]), float(best_position[1]), best_value

In [ ]:






def teaching_learning_optimization(func, rnge, pop_size=30, generations=50):
    lower, upper = rnge
    dim = 2
    positions = np.random.uniform(lower, upper, size=(pop_size, dim))
    fitness = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, positions)
    best_idx = np.argmin(fitness)
    best_position = positions[best_idx].copy()
    best_value = float(fitness[best_idx])

    for _ in range(generations):
        mean_vector = positions.mean(axis=0)
        teacher_idx = np.argmin(fitness)
        teacher = positions[teacher_idx].copy()
        TF = np.random.randint(1, 3)
        for i in range(pop_size):
            r = np.random.rand(dim)
            candidate = positions[i] + r * (teacher - TF * mean_vector)
            candidate = np.clip(candidate, lower, upper)
            candidate_value = float(func(tuple(candidate)))
            if candidate_value < fitness[i]:
                positions[i] = candidate
                fitness[i] = candidate_value
                if candidate_value < best_value:
                    best_value = candidate_value
                    best_position = candidate.copy()
        mean_vector = positions.mean(axis=0)
        teacher_idx = np.argmin(fitness)
        teacher = positions[teacher_idx].copy()
        for i in range(pop_size):
            j = np.random.randint(pop_size - 1)
            if j >= i:
                j += 1
            Xi = positions[i]
            Xj = positions[j]
            r = np.random.rand(dim)
            if fitness[i] < fitness[j]:
                candidate = Xi + r * (Xi - Xj)
            else:
                candidate = Xi + r * (Xj - Xi)
            candidate = np.clip(candidate, lower, upper)
            candidate_value = float(func(tuple(candidate)))
            if candidate_value < fitness[i]:
                positions[i] = candidate
                fitness[i] = candidate_value
                if candidate_value < best_value:
                    best_value = candidate_value
                    best_position = candidate.copy()
        current_best_idx = np.argmin(fitness)
        if fitness[current_best_idx] < best_value:
            best_value = float(fitness[current_best_idx])
            best_position = positions[current_best_idx].copy()

    return float(best_position[0]), float(best_position[1]), best_value


In [ ]:
ALGORITHMS = {
    "Differential Evolution": differential_evolution,
    "Particle Swarm Optimization": particle_swarm_optimization,
    "SOMA All-to-One": soma_all_to_one,
    "Firefly Algorithm": firefly_algorithm,
    "Teaching-Learning Optimization": teaching_learning_optimization,
}


def run_all_experiments(test_functions, algorithms, runs=10, base_seed=1234):
    results = {}
    for func_idx, (func, bounds) in enumerate(test_functions.items()):
        lower, upper, _ = bounds
        function_results = {}
        for algo_idx, (algo_name, algo_fn) in enumerate(algorithms.items()):
            rows = []
            for run in range(runs):
                seed = base_seed + func_idx * 10000 + algo_idx * 1000 + run
                np.random.seed(seed)
                best_x, best_y, best_value = algo_fn(func, (lower, upper))
                rows.append(
                    {
                        "run": run + 1,
                        "best_value": best_value,
                        "best_x": best_x,
                        "best_y": best_y,
                    }
                )
            df_runs = pd.DataFrame(rows)
            stats_mean = df_runs[["best_value", "best_x", "best_y"]].mean()
            stats_std = df_runs[["best_value", "best_x", "best_y"]].std(ddof=1)
            summary = pd.DataFrame(
                [
                    {"statistic": "mean", **stats_mean.to_dict()},
                    {"statistic": "std", **stats_std.to_dict()},
                ]
            )
            df_runs_formatted = df_runs.copy()
            df_runs_formatted["statistic"] = df_runs_formatted["run"].apply(lambda r: f"run_{r:02d}")
            df_runs_formatted = df_runs_formatted.drop(columns="run")
            df_result = pd.concat([summary, df_runs_formatted], ignore_index=True)
            function_results[algo_name] = df_result[["statistic", "best_value", "best_x", "best_y"]]
        results[func.__name__] = function_results
    return results


In [ ]:
experiment_results = run_all_experiments(TEST_FUNCTIONS, ALGORITHMS, runs=10, base_seed=1234)


In [ ]:
for function_name, algo_tables in experiment_results.items():
    print(f"=== Function: {function_name} ===")
    for algo_name, df_result in algo_tables.items():
        print(f"Algorithm: {algo_name}")
        display(df_result)
    print()
